In [1]:
import requests
import util.item_analysis as ui
from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
# 예
market = ['심리 상담 센터']
addr = '서울시 영등포구 여의도동'

# 고객이 입력한 상품을 파는 매장을 리스트로 가져오기
market_list = market

In [2]:
# 고객이 입력한 주소 ['시', '구'] 또는 ['군', '면', '리'] 가져오기
addr_list = addr.split(' ')
addr_list1 = []
addr_list2 = []

for i in addr_list:
    if i[-1] == '시' or i[-1] == '구' or i[-1] == '동':
        addr_list1.append(i)
    elif i[-1] == '군' or i[-1] == '면' or i[-1] == '리':
        addr_list2.append(i)

addr1 = ' '.join(addr_list1)
addr2 = ' '.join(addr_list2)

if addr1 != '':
    addr_result = addr1
else:
    addr_result = addr2

addr_result

'서울시 영등포구 여의도동'

In [3]:
# 고객이 입력한 주소 좌표 구하기
try:
    with open('keys/카카오api.txt') as file:
        kakao_key = file.read()

    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(addr)}'
    result = requests.get(url, headers=header).json()
    lat = float(result['documents'][0]['y'])
    lng = float(result['documents'][0]['x'])
except:
     print('주소 형식이 올바르지 않습니다.')

In [4]:
lat, lng

(37.5267849150832, 126.930167091136)

In [18]:
# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

In [19]:
# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

In [20]:
# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)

In [8]:
# '상세 주소 화살표' 클릭
try:
    juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
    juso_way[0].click()
    time.sleep(1)
except:
    print('근처에 매장이 없습니다')

In [9]:
# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source)
market_list = soup.find('ul', class_='eDFz9')
mart = market_list.find('li', recursive=False)
mart

<li class="VLTHu" data-laim-exp-id="undefined"><div class="qbGlu"><div class="TTfa9"><a class="place_thumb" href="https://m.place.naver.com/place/1489710778/photo?entry=ple" role="button"><div class="lazyload-wrapper"><img alt="여의도심리상담센터" data-grid-lazy="false" height="88" src="https://search.pstatic.net/common/?autoRotate=true&amp;quality=95&amp;type=f160_160&amp;src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220824_292%2F1661310463886veLI9_JPEG%2F20220824_111349.jpg" width="88"/></div><span class="place_thumb_count"><span class="place_blind">이미지수</span>5</span></a></div><div class="ouxiq"><a class="P7gyV" href="https://m.place.naver.com/place/1489710778?entry=ple" role="button"><div class="ApCpt"><div class="place_bluelink C6RjW"><span class="YwYLL">여의도심리상담센터</span><span class="YzBgS">심리상담</span></div></div></a><a class="P7gyV" href="https://m.place.naver.com/place/1489710778?entry=ple" role="button"><div class="iHXbN"><div class="Gvf9B"><span class="XP3ml yTY83">영업 중</span><span class=

In [10]:
market_list.select_one('div.AbTyi').get_text().split('복사')[1][2:]

'서울특별시 영등포구 여의도동 25-3 동화빌딩 6층 605호'

In [41]:
center.select_one('span.JsCty > a').get('href')[4:]

'02-6274-0030'

In [42]:
results = []

# 네이버 플레이스 셀레니움으로 들어가기
n_place_url = f'https://m.place.naver.com/place/list?x={lng}&y={lat}&query={market[0]} {addr}'
driver = webdriver.Chrome()
driver.get(n_place_url)

# '목록보기' 클릭
filter = driver.find_element(By.XPATH, '//*[@id="_place_portal_root"]/div/a') 
filter.click()
time.sleep(3)

# '거리순' 클릭
short_way = driver.find_element(By.XPATH, '//*[@id="_list_scroll_container"]/div/div/div[1]/div/div/div/span[2]/a') 
short_way.click()
time.sleep(3)



# 마켓 정보 불러오기
soup = BeautifulSoup(driver.page_source, 'html.parser')
center_list = soup.find('ul', class_='eDFz9')

for i in range(5):
    if center_list:
        center = center_list.select('li', recursive=False)[i]

        if center:
            # 현재 위치에서 마켓 거리
            c_distance_element = center.select_one('span.lWwyx.NVngW')
            c_distance = c_distance_element.get_text().split('서')[1] if c_distance_element else 'N/A'

            # 마켓 이름
            if center.select_one('span.YwYLL'):
                c_title = center.select_one('span.YwYLL').get_text() 
            else:
                c_title = 'N/A'   

            # '상세 주소 화살표' 클릭
            try:
                juso_way = driver.find_elements(By.CLASS_NAME, 'uFxr1')
                juso_way[i].click()
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                center_list = soup.find('ul', class_='eDFz9')
                # 마켓 도로명 주소
                if center_list.select_one('div.zZfO1'):
                    if center_list.select_one('div.zZfO1').get_text()[:2] == '도로':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[0][3:]
                    elif center_list.select_one('div.zZfO1').get_text()[:2] == '지번':
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                    else:
                        c_addr = center_list.select_one('div.zZfO1').get_text().split('복사')[1][2:]
                else:
                    c_addr = 'N/A' 
                
                # 전화번호 추출
                if center.select_one('span.JsCty > a'):    
                    c_phone = center.select_one('span.JsCty > a').get('href')[4:]
                else:
                    c_addr = 'N/A' 

                

                
                time.sleep(1)
            except:
                print('근처에 매장이 없습니다')
                
            # 딕셔너리 형태로 저장
            center_data = {
                '거리': c_distance,
                '매장명': c_title,
                '주소': c_addr,
                '전화번호': c_phone
            }
            # 딕셔너리 형태로 저장한 것을 리스트에 저장 (3가지 마켓을 넣어야 하므로)
            results.append(center_data)

        else:
            print('마켓 정보를 찾을 수 없습니다.')
    else:
        print('마켓 리스트를 찾을 수 없습니다.')

results_df = pd.DataFrame(results)
print(results_df)


     거리                 매장명                                       주소  \
0  430m           여의도심리상담센터         서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호   
1  750m     마음iN심리상담센터 여의도점              서울특별시 영등포구 여의나루로 42 5층 510호   
2  780m         이주은 부부상담 본점           서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어   
3  830m                조이에듀  서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)   
4  960m  허그맘허그인심리상담센터 여의도센터  서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호   

             전화번호  
0    02-6274-0030  
1  0507-1493-3030  
2   010-9070-0153  
3  0507-1354-1151  
4    02-2135-7542  


In [ ]:
center_list.select_one('div.gkDbw')

In [43]:
df = pd.DataFrame(results)
df

,거리,매장명,주소,전화번호
0,430m,여의도심리상담센터,서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호,02-6274-0030
1,750m,마음iN심리상담센터 여의도점,서울특별시 영등포구 여의나루로 42 5층 510호,0507-1493-3030
2,780m,이주은 부부상담 본점,서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어,010-9070-0153
3,830m,조이에듀,"서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)",0507-1354-1151
4,960m,허그맘허그인심리상담센터 여의도센터,서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호,02-2135-7542


In [44]:
# 고객이 입력한 주소 좌표 구하기
from tqdm import tqdm
with open('keys/카카오api.txt') as file:
    kakao_key = file.read()

base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}
lat_list, lng_list = [], []

for i in tqdm(df.index):
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    lat_list.append(float(result['documents'][0]['y']))
    lng_list.append(float(result['documents'][0]['x']))


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:00<00:00, 15.53it/s]


In [67]:
df['위도'] = lat_list
df['경도'] = lng_list
df

,거리,매장명,주소,전화번호,위도,경도
0,430m,여의도심리상담센터,서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호,02-6274-0030,37.523781,126.926874
1,750m,마음iN심리상담센터 여의도점,서울특별시 영등포구 여의나루로 42 5층 510호,0507-1493-3030,37.521470,126.924971
2,780m,이주은 부부상담 본점,서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어,010-9070-0153,37.520241,126.926407
3,830m,조이에듀,"서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)",0507-1354-1151,37.519365,126.929590
4,960m,허그맘허그인심리상담센터 여의도센터,서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호,02-2135-7542,37.518189,126.931355


In [68]:
df.to_csv('data/상담센터.csv')


In [84]:
df = pd.read_csv('data/상담센터.csv')
df.head()

,Unnamed: 0,거리,매장명,주소,전화번호,위도,경도
0,0,430m,여의도심리상담센터,서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호,02-6274-0030,37.523781,126.926874
1,1,750m,마음iN심리상담센터 여의도점,서울특별시 영등포구 여의나루로 42 5층 510호,0507-1493-3030,37.521470,126.924971
2,2,780m,이주은 부부상담 본점,서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어,010-9070-0153,37.520241,126.926407
3,3,830m,조이에듀,"서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)",0507-1354-1151,37.519365,126.929590
4,4,960m,허그맘허그인심리상담센터 여의도센터,서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호,02-2135-7542,37.518189,126.931355


In [85]:
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df

,거리,매장명,주소,전화번호,위도,경도
0,430m,여의도심리상담센터,서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호,02-6274-0030,37.523781,126.926874
1,750m,마음iN심리상담센터 여의도점,서울특별시 영등포구 여의나루로 42 5층 510호,0507-1493-3030,37.521470,126.924971
2,780m,이주은 부부상담 본점,서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어,010-9070-0153,37.520241,126.926407
3,830m,조이에듀,"서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)",0507-1354-1151,37.519365,126.929590
4,960m,허그맘허그인심리상담센터 여의도센터,서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호,02-2135-7542,37.518189,126.931355


In [86]:
df = df.astype({'위도':'float', '경도':'float'})
df

,거리,매장명,주소,전화번호,위도,경도
0,430m,여의도심리상담센터,서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호,02-6274-0030,37.523781,126.926874
1,750m,마음iN심리상담센터 여의도점,서울특별시 영등포구 여의나루로 42 5층 510호,0507-1493-3030,37.521470,126.924971
2,780m,이주은 부부상담 본점,서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어,010-9070-0153,37.520241,126.926407
3,830m,조이에듀,"서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)",0507-1354-1151,37.519365,126.929590
4,960m,허그맘허그인심리상담센터 여의도센터,서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호,02-2135-7542,37.518189,126.931355


In [87]:
df.head()

,거리,매장명,주소,전화번호,위도,경도
0,430m,여의도심리상담센터,서울특별시 영등포구 여의나루로 71 동화빌딩 6층 605호,02-6274-0030,37.523781,126.926874
1,750m,마음iN심리상담센터 여의도점,서울특별시 영등포구 여의나루로 42 5층 510호,0507-1493-3030,37.521470,126.924971
2,780m,이주은 부부상담 본점,서울특별시 영등포구 의사당대로 127 롯데캐슬 엠파이어,010-9070-0153,37.520241,126.926407
3,830m,조이에듀,"서울특별시 영등포구 여의대방로65길 13 602호(여의도동, 유창빌딩)",0507-1354-1151,37.519365,126.929590
4,960m,허그맘허그인심리상담센터 여의도센터,서울특별시 영등포구 여의대방로 376 나라키움여의도빌딩 305~307호,02-2135-7542,37.518189,126.931355


In [76]:
import folium
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=13)
map

In [88]:
df.columns

Index(['거리', '매장명', '주소', '전화번호', '위도', '경도'], dtype='object')

In [79]:
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=16)
for i in df.index:
    folium.Marker([df.위도[i], df.경도[i]], tooltip= f'<strong>{df.매장명[i]}</strong><br>{df.주소[i]}<br>{df.전화번호[i]}').add_to(map)

map


In [89]:
def map_by_dong(df, dong):
    gf = df[df.BJDONG_NM.str.find(dong) == 0.0] 
    map = folium.Map([gf.위도.mean(), gf.경도.mean()], zoom_start=15)
    for i in gf.index:
        folium.Marker([df.위도[i], df.경도[i]], tooltip= [df.매장명[i], '<br>', df.주소[i]]).add_to(map)

    map.save('dong.html')

    map

In [90]:
df.columns

Index(['거리', '매장명', '주소', '전화번호', '위도', '경도'], dtype='object')

In [91]:
import folium
import pandas as pd

# 가상의 DataFrame을 생성하고 예시 데이터로 채웁니다.
center_data = {
                '거리': c_distance,
                '매장명': c_title,
                '주소': c_addr,
                '전화번호': c_phone
            }

# 지도 생성
map = folium.Map([df.위도.mean(), df.경도.mean()], zoom_start=14)

# DataFrame의 각 행에 대해 마커를 추가합니다.
for i in df.index:
    folium.Marker([df.위도[i], df.경도[i]], tooltip=[df.매장명[i], '<br>', df.주소[i]]).add_to(map)

# 지도 저장
map.save('map.html')


In [92]:
map